In [19]:
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("dev.langchain4j:langchain4j:1.0.0-beta2")
        implementation("dev.langchain4j:langchain4j-mcp:1.0.0-beta2")
        implementation("dev.langchain4j:langchain4j-vertex-ai-gemini:1.0.0-beta2")
    }
}
// list the library, if the library is not exist, restart kernel
notebook.currentClasspath.joinToString("\n")

/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-392/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-stdlib-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-392/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-reflect-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-392/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlinx-serialization-core-jvm-1.6.3.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-392/kotlin-jupyter-script-classpath-shadowed-zip_extracted/annotations-13.0.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-392/kotlin-jupyter-sc

In [15]:
val projectId = System.getenv("GOOGLE_CLOUD_PROJECT") ?: "gaplotech" // Replace with your actual project ID
val location = System.getenv("GOOGLE_CLOUD_REGION") ?: "us-central1"

print(projectId to location)

(gaplotech, us-central1)

In [18]:
// Define the model to use
val modelName = "gemini-2.0-flash-exp"


In [17]:
import dev.langchain4j.mcp.McpToolProvider
import dev.langchain4j.mcp.client.DefaultMcpClient
import dev.langchain4j.mcp.client.McpClient
import dev.langchain4j.mcp.client.transport.http.HttpMcpTransport
import dev.langchain4j.model.vertexai.VertexAiGeminiChatModel
import dev.langchain4j.service.AiServices
import dev.langchain4j.service.SystemMessage
import dev.langchain4j.service.UserMessage
import dev.langchain4j.service.V

interface Supervisor {
    @SystemMessage("You are a helpful ai assistant with a list of tools. Use avaliable tools automatically as parallel as possible.")
    @UserMessage("{{message}}")
    fun chat(@V("message") userMessage: String): String
}

val transport = HttpMcpTransport.Builder()
    .sseUrl("http://localhost:3001/sse")
    .build()

val mcpClient: McpClient = DefaultMcpClient.Builder()
    .transport(transport)
    .build()

val toolProvider = McpToolProvider.builder()
    .mcpClients(listOf(mcpClient))
    .build();

val model = VertexAiGeminiChatModel.builder()
    .project(projectId)
    .location(location)
    .modelName(modelName)
    .build()

val bot = AiServices.builder(Supervisor::class.java)
    .chatLanguageModel(model)
    .toolProvider(toolProvider)
    .build()

print(bot.chat("get bitcoin print today. use avaliable tool automatically"))

The current price of Bitcoin is $82434.43.
